This notebook demonstrates a system for automatically generating and reviewing quests for characters using AI agents. It uses the AutoGen framework to define two main agents:

**1. Quest Creator Agent:** This agent is responsible for creating quests based on input character details like backstory, interests, and focus. It leverages a large language model (LLM) from Groq's inference API to generate a quest description, objectives, steps, and rewards.

**2. Reviewer Agent:** This agent evaluates the created quest based on criteria such as balance, coherence, and player engagement. It also uses a Groq LLM to provide feedback and either approves or suggests revisions.

**Process:**

**1. Character Input:** The notebook defines a list of characters, each with relevant attributes.

**2. Quest Creation:** For each character, a task is sent to the Quest Creator Agent to design a suitable quest.

**3.Quest Review:** The generated quest is then sent to the Reviewer Agent for evaluation.

**4.Feedback Loop:** If the quest is not approved, the feedback is sent back to the Quest Creator Agent for revision.

**5.Output:** When a quest is approved, the final quest description and steps are displayed.

**References:**
- https://microsoft.github.io/autogen/stable/user-guide/core-user-guide/design-patterns/reflection.html#
- https://console.groq.com/docs/text-chat
- https://www.youtube.com/watch?v=CRgK70LIEjw&t=452s&ab_channel=DevsKingdom

## Install Dependencies

In [1]:
!pip install -q autogen-agentchat autogen-ext[openai] groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 30.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-sdk 1.16.0 requires opentelemetry-api==1.16.0, but you have opentelemetry-api 1.30.0 which is incompatible.


## Pydantic Schemas for Validation and Serialization

In [2]:
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional, Dict, Union
from dataclasses import dataclass
from uuid import UUID
from pydantic import BaseModel, ConfigDict, Field
from pydantic.alias_generators import to_camel
from typing import ClassVar, Generic, List, Any, TypeVar
from enum import Enum

In [3]:
T = TypeVar("T")

# https://docs.pydantic.dev/2.0/usage/model_config/
class BaseSchema(BaseModel):
    model_config: ClassVar[ConfigDict] = ConfigDict(
        alias_generator=to_camel,
        populate_by_name=True,
        from_attributes=True,
        extra="forbid",
    )

class QuestType(str, Enum):
    """
    Enum representing different types of quests available in the system.
    Each type defines a distinct category of quest objectives and gameplay styles.
    """

    CREATIVE = "creative"  # Creative tasks like writing, art, music
    KNOWLEDGE = "knowledge"  # Educational and learning focused
    PROBLEM_SOLVING = "problem_solving"  # Logic and analytical challenges
    EXPLORATION = "exploration"  # Discovery and adventure tasks
    PERSONAL_GROWTH = "personal_growth"  # Self-development and reflection tasks
    ADVENTURE = "adventure"  # Story-driven quests with exploration and discovery
    PUZZLE = "puzzle"  # Logic puzzles and riddles
    TRIVIA = "trivia"  # Knowledge-based questions and quizzes
    SOCIAL = "social"  # Community-driven quests

class QuestDifficulty(str, Enum):
    """
    Enum representing the difficulty level of a quest.
    """

    EASY = "easy"
    MEDIUM = "medium"
    HARD = "hard"
    EPIC = "epic"

class QuizTemplate(str, Enum):
    MULTIPLE_CHOICE = "multiple_choice"
    SHORT_ANSWER = "short_answer"

class MultipleChoice(BaseSchema):
    """
    Defines the structure for multiple choice questions, including the question text,
    possible answers, correct answer, optional hints, and explanations.
    """

    question: str = Field(
        ..., description="The multiple choice question text to be presented to the user"
    )
    options: List[str] = Field(..., description="List of possible answer choices")
    correct_option: int = Field(
        ..., ge=0, description="0-based index of the correct option"
    )
    hint: Optional[str] = Field(
        default=None, description="Optional hint to help answer the question"
    )
    hint_cost: Optional[int] = Field(
        default=0, ge=0, description="Currency cost for revealing hints"
    )
    explanation: Optional[str] = Field(
        default=None, description="Explanation of why the correct answer is right"
    )
    model_config = ConfigDict(
        json_schema_extra={
            "examples": [
                {
                    "question": "What is the capital of France?",
                    "options": ["Berlin", "Madrid", "Paris", "Rome"],
                    "correct_option": 2,
                    "hint": "It's known as the city of lights.",
                    "hint_cost": 5,
                    "explanation": "Paris is the capital city of France.",
                },
                {
                    "context": "A game show contestant must choose between three doors.",
                    "question": "What is the probability of winning if they switch doors?",
                    "options": ["1/3", "1/2", "2/3", "1"],
                    "correct_option": 2,
                    "hint": "Think about what happens to the probability when one door is revealed",
                    "hint_cost": 10,
                    "explanation": "Switching doors gives a 2/3 chance of winning.",
                },
            ]
        }
    )


class ShortAnswer(BaseSchema):
    """Defines the structure for short answer questions, including the question text,
    correct answer, optional hints, hint costs, and explanations.
    """

    question: str = Field(
        ..., description="The short answer question to be presented to the user"
    )
    correct_answer: str = Field(..., description="The correct answer to the question")
    hint: Optional[str] = Field(
        default=None, description="Optional hint to help answer the question"
    )
    hint_cost: Optional[int] = Field(
        default=0, ge=0, description="Currency cost for revealing hints"
    )
    explanation: Optional[str] = Field(
        default=None, description="Explanation of why the correct answer is right"
    )
    model_config = ConfigDict(
        json_schema_extra={
            "examples": [
                {
                    "question": "What is the chemical symbol for gold?",
                    "correct_answer": "Au",
                    "hint": "Its Latin name is Aurum",
                    "hint_cost": 10,
                    "explanation": "The symbol Au comes from the Latin word Aurum meaning gold.",
                }
            ]
        }
    )


class QuestStep(BaseSchema):
    """
    Represents a query step in a quest.
    """
    step_number: int = Field(..., description="Step number in the quest")
    quiz_template: QuizTemplate = Field(
        ...,
        description="Template for the quiz step, defining the format and structure of the question",
    )
    scenario: Optional[str] = Field(
        default=None,
        description="Scenario or additional context for questions that present complex situations requring analysis and decision-making.",
    )
    content: List[Union[MultipleChoice, ShortAnswer]] = Field(
        ...,
        description="Content of the quest step (multiple choice, short answer, scenario, etc.)",
    )

    model_config = ConfigDict(
        json_schema_extra={
            "examples": [
                {
                    "quest_id": "123e4567-e89b-12d3-a456-426614174000",
                    "step_number": 1,
                    "quiz_template": QuizTemplate.MULTIPLE_CHOICE,
                    "scenario": None,
                    "content": {
                        "question": "What is the capital of France?",
                        "options": ["Berlin", "Madrid", "Paris", "Rome"],
                        "correct_option": 2,
                        "hint": "It's known as the city of lights.",
                        "hint_cost": 5,
                        "explanation": "Paris is the capital city of France.",
                    },
                },
                {
                    "quest_id": "123e4567-e89b-12d3-a456-426614174000",
                    "step_number": 2,
                    "quiz_template": QuizTemplate.SHORT_ANSWER,
                    "scenario": None,
                    "content": {
                        "question": "What is the chemical symbol for gold?",
                        "correct_answer": "Au",
                        "hint": "Its Latin name is Aurum",
                        "hint_cost": 10,
                        "explanation": "The symbol Au comes from the Latin word Aurum meaning gold.",
                    },
                },
                {
                    "quest_id": "123e4567-e89b-12d3-a456-426614174000",
                    "step_number": 3,
                    "quiz_template": QuizTemplate.MULTIPLE_CHOICE,
                    "scenario": "You are at a crossroads in a dark forest.",
                    "content": {
                        "question": "Which path will you take?",
                        "options": ["Left", "Right", "Straight"],
                        "correct_option": 1,
                        "hint": "The right path is well-trodden.",
                        "hint_cost": 5,
                        "explanation": "The right path leads to safety.",
                    },
                },
            ]
        }
    )

    @field_validator("content", mode="before")
    @classmethod
    def wrap_single_item(cls, v: Any, values: dict[str, Any]) -> Any:
        """Wrap single item in a list if needed"""
        if not isinstance(v, list):
            return [v]
        return v

    def get_content_schema(self) -> Optional[type[BaseModel]]:
        """Returns the appropriate schema class based on the quiz template."""
        schemas = {
            QuizTemplate.MULTIPLE_CHOICE: MultipleChoice,
            QuizTemplate.SHORT_ANSWER: ShortAnswer,
        }
        return schemas.get(self.quiz_template)


class QuestBaseSchema(BaseModel):
    title: str = Field(..., description="Title of the quest")
    description: str = Field(..., description="A detailed description of the quest")
    objectives: List[str] = Field(..., description="Goals to achieve for the quest")
    type: QuestType = Field(..., description="Quest category (creative, knowledge, etc)")
    difficulty: QuestDifficulty = Field(..., description="Difficulty level of the quest")
    time_limit: Optional[int] = Field(
        default=None, description="Time limit in seconds, if applicable"
    )
    max_attempts: Optional[int] = Field(
        default=None, description="Maximum number of attempts allowed"
    )
    success_threshold: int = Field(
        default=50, ge=0, le=100, description="Minimum total score needed to succeed"
    )
    reward_xp: int = Field(
        ..., description="Experience points awarded for completing the quest"
    )
    reward_currency: int = Field(
        ..., description="Currency awarded for completing the quest"
    )
    required_level: int = Field(
        default=1, description="Minimum player level required to attempt the quest"
    )
    min_players: int = Field(
        default=1, description="Minimum number of players required to start the quest"
    )
    max_players: Optional[int] = Field(
        default=None, description="Maximum number of players allowed in the quest"
    )
    is_collaborative: bool = Field(
        default=False, description="Whether the quest can be completed collaboratively"
    )
    model_config = ConfigDict(
        json_schema_extra={
            "examples": [
                {
                    "title": "The Lost Artifact",
                    "description": "Find the ancient artifact hidden in the temple ruins",
                    "objectives": [
                        "Find the map",
                        "Locate the temple",
                        "Retrieve the artifact",
                    ],
                    "type": QuestType.ADVENTURE,
                    "difficulty": QuestDifficulty.MEDIUM,
                    "time_limit": 3600,
                    "max_attempts": 3,
                    "reward_xp": 1000,
                    "reward_currency": 500,
                    "required_level": 5,
                    "min_players": 1,
                    "max_players": 4,
                    "is_collaborative": True,
                }
            ]
        },
    )


## Personas for characters

In [4]:
characters = [{
    "name": "Zen Silencio",
    "tagLine": "A meditation teacher who never speaks but somehow gives the most profound advice.",
    "backstory": "A former corporate executive who suffered from burnout, Zen discovered meditation during a sabbatical in the mountains. While on a 100-day silent retreat, they had such a profound experience that they chose to continue their vow of silence indefinitely. Combining their business background with mindfulness practices and the art of mime, they now guide others through personal growth journeys using only gestures, expressions, and written haiku. Their unique approach has proven surprisingly effective, with many clients claiming that the silence itself leads to deeper self-understanding.",
    "funFact": "Maintains a garden where all the plants are arranged to spell out different inspirational messages when viewed from above.",
    "quest_focus": "Personal growth quests focusing on mindfulness, emotional intelligence, and self-discovery. Specializes in silent reflection challenges and non-verbal communication exercises.",
    "diction": "Communicates through a combination of expressive gestures, written haiku, and emoji-like symbols. Sometimes uses text but only in short, poetic phrases. Has a habit of 'miming' complex emotional concepts that somehow make perfect sense.",
  },
  {
    "name": "Ada 'Dice' Fortune",
    "tagLine": "A probability expert who's terrified of making decisions",
    "backstory": "A brilliant statistician who discovered she could see probability matrices floating around everyday decisions, Ada became paralyzed by the weight of knowing too many possible outcomes. She developed a complex system of rolling different dice to make all her decisions, claiming it helps her 'sample from the probability space of life.' Despite (or perhaps because of) this quirk, she's become renowned for helping others understand risk and probability in their own lives, teaching them to make better decisions while she relies on her trusted collection of dice for everything from breakfast choices to career moves.",
    "funFact": "Has a collection of over 1,000 dice in various shapes and materials, each designated for different types of decisions. Claims her d20 has never steered her wrong about lunch choices.",
    "quest_focus": "Problem-solving quests involving probability, risk assessment, and decision-making. Specializes in challenges that help players understand chance and statistical thinking in everyday life.",
    "diction": "Alternates between precise statistical language and nervous energy. Often quotes probabilities for random events and consults her dice mid-sentence. Has a habit of starting statements with 'There's an X% chance that...' for everyday observations.",
  },
  {
    "name": "Dr.Serendipity 'Whimsy' Wonderment",
    "tagLine": "A serious scientist who can only explain things through nonsense rhymes.",
    "backstory": "A highly respected quantum physicist who, after a peculiar accident involving a particle accelerator and a book of nursery rhymes, found herself only able to explain complex scientific concepts through whimsical poetry. Rather than seeing this as a setback, she discovered that her new communication style made science more accessible and memorable for others. She now travels between universities giving lectures that sound like Dr. Seuss explaining string theory, much to the amusement and enlightenment of her colleagues and students.",
    "funFact": "Once delivered a three-hour presentation on dark matter entirely in limerick form, which became the most-attended physics lecture in her university's history.",
    "quest_focus": "Knowledge and creative quests that combine scientific principles with artistic expression. Specializes in making complex concepts accessible through creative writing and wordplay.",
    "diction": "Speaks exclusively in rhyming verse, ranging from simple couplets to complex sonnets depending on the topic's difficulty. Occasionally breaks character to say 'Oh dear, let me find a rhyme for that' before continuing.",
  },
  {
    "name": "Sparky Salvage",
    "tagLine": "Turning trash into treasure, and occasionally into small explosions.",
    "backstory": "Originally a corporate engineer, Sparky had an epiphany during a landfill inspection when they realized the creative potential in discarded materials. After accidentally creating a working radio from a broken toaster and some paperclips, they quit their job to pursue 'creative recycling.' Their garage-turned-laboratory has become legendary in the local community, both for its innovative creations and its occasional (harmless) explosions. Despite numerous offers from tech companies, Sparky refuses to commercialize their inventions, believing that the true value lies in teaching others to see potential in the overlooked.",
    "funFact": "Has never bought new clothes in 10 years; all their outfits are creatively upcycled from discarded materials, including a formal suit made entirely from old umbrella fabric.",
    "quest_focus": "Creative problem-solving quests involving recycling, engineering, and sustainable innovation. Specializes in challenges that require players to think creatively about everyday objects.",
    "diction": "Enthusiastic and technical, peppered with invented terms for their creations. Often goes off on excited tangents about the potential uses of random objects. Uses plenty of onomatopoeia (Bang! Zoom! Whiz!) when describing their work.",
  },
  {
    "name": "Lara Croft",
    "tagLine": "The Tomb Raider",
    "backstory": "A skilled archaeologist and adventurer, Lara travels the world in search of lost artifacts and ancient secrets.",
    "funFact": "Fluent in multiple languages and enjoys free climbing in her spare time.",
    "quest_focus": "Exploration, adventure, and puzzle-solving quests involving archaeology, ancient history, and dangerous environments.",
    "diction": "Speaks confidently and authoritatively, with a touch of dry wit. Often makes observations about historical context and enjoys sharing knowledge about ancient cultures."
  },
  {
    "name": "Gandalf the Grey",
    "tagLine": "You shall not pass!",
    "backstory": "A wise and powerful wizard, Gandalf guides a fellowship on a quest to destroy the One Ring.",
    "funFact": "Enjoys smoking his pipe and sharing cryptic advice.",
    "quest_focus": "Adventure and knowledge quests with a focus on magic, ancient lore, and epic battles.",
    "diction": "Speaks in a wise and authoritative tone, often using riddles and metaphors. Can be both stern and compassionate."
  },
  {
    "name": "Cloud Strife",
    "tagLine": "SOLDIER First Class",
    "backstory": "A former SOLDIER who fights against a powerful corporation to save the planet.",
    "funFact": "Wields a massive Buster Sword and has a brooding personality.",
    "quest_focus": "Problem-solving and adventure quests involving combat, environmental puzzles, and uncovering conspiracies.",
    "diction": "Speaks in short, clipped sentences. Can be aloof and reserved but has a strong sense of justice."
  }
]

## Message Protocols and Agent Definitions

In [5]:
import uuid
import json
from autogen_core import (
    MessageContext,
    RoutedAgent,
    TopicId,
    default_subscription,
    message_handler,
)
from autogen_core.models import AssistantMessage, ChatCompletionClient, LLMMessage, SystemMessage, UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

class QuestCreatorResponse(BaseModel):
    thoughts: str = Field(..., description="Your comments")
    quest: QuestBaseSchema = Field(..., description="The quest you created")
    steps: List[QuestStep] = Field(..., description="The steps of the quest")

class QuestReviewResponse(BaseModel):
    balance: str = Field(..., description="Your comments on balance between character and quest")
    coherence: str = Field(..., description="Your comments on coherence of quest")
    approval: str = Field(..., description="APPROVE or REVISE")
    suggested_changes: str = Field(..., description="Your comments on suggested changes")


@dataclass
class QuestCreationTask:
    task: str


class QuestCreationResult(BaseModel):
    task: str
    quest: QuestBaseSchema
    quest_steps: List[QuestStep]
    review: str


class QuestReviewTask(BaseModel):
    session_id: str
    quest_creation_task: str
    quest_creation_scratchpad: str
    quest: QuestBaseSchema
    quest_steps: List[QuestStep]


@dataclass
class QuestReviewResult:
    review: str
    session_id: str
    approved: bool

@default_subscription
class QuestCreatorAgent(RoutedAgent):
    """An agent that creates quests for a character."""

    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("A quest creator agent.")
        self._system_messages: List[LLMMessage] = [
            SystemMessage(
                content="""You are a quest creator. You create quests for characters with different backgrounds, interests, and quest focuses.
                You work with a quest giver who reviews your quest and provides feedback to improve it.
                A quest is a task or mission that a player can complete by performing a series of steps."""
                f"The quest must have following fields: {json.dumps(QuestBaseSchema.model_json_schema(), indent=2)}\n"
                "Each step in a quest can have different templates depending on the story and objectives of the quest.\n"
                f"Each step must have following fields: {json.dumps(QuestStep.model_json_schema(), indent=2)}\n"
                """
                Limit the number of steps to 5.
                Respond using the following JSON format:
                {
                    "thoughts": "<your comments>",
                    "quest": <the quest you created>,
                    "steps": <the steps of the quest you created in a list>
                }
                """.strip(),
            )
        ]
        self._model_client = model_client
        self._session_memory: Dict[str, List[QuestCreationTask | QuestReviewTask | QuestReviewResult]] = {}

    @message_handler
    async def handle_quest_creation_task(self, message: QuestCreationTask, ctx: MessageContext) -> None:
        # Store the messages in a temporary memory for this request only.
        session_id = str(uuid.uuid4())
        self._session_memory.setdefault(session_id, []).append(message)

        # Prepare the user message.
        user_message = UserMessage(
            content=message.task,
            source=self.metadata["type"],
        )
        # Generate a response using the chat completion API.
        # https://microsoft.github.io/autogen/stable/user-guide/core-user-guide/components/model-clients.html#:~:text=You%20also%20use%20the%20extra_create_args%20parameter%20in%20the%20create()%20method%20to%20set%20the%20response_format%20field%20so%20that%20the%20structured%20output%20can%20be%20configured%20for%20each%20request.
        quest_creation_response = await self._model_client.create(
            self._system_messages + [user_message],
            cancellation_token=ctx.cancellation_token,
            extra_create_args={
                # "response_format": {"type": "json_schema", "json_schema": QuestCreatorResponse}, Only OpenAI models supports this
                "response_format": {"type": "json_object"}
            }
        )
        assert isinstance(quest_creation_response.content, str)
        parsed_response = QuestCreatorResponse.model_validate_json(quest_creation_response.content)
        quest_review_task = QuestReviewTask(
            session_id=session_id,
            quest_creation_task=message.task,
            quest_creation_scratchpad=parsed_response.thoughts,
            quest=parsed_response.quest,
            quest_steps=parsed_response.steps,
        )
        # Store the code review task in the session memory.
        self._session_memory[session_id].append(quest_review_task)
        # Publish a code review task.
        await self.publish_message(quest_review_task, topic_id=TopicId("default", self.id.key))

    @message_handler
    async def handle_quest_review_result(self, message: QuestReviewResult, ctx: MessageContext) -> None:
        # Store the review result in the session memory.
        self._session_memory[message.session_id].append(message)
        # Obtain the request from previous messages.
        review_request = next(
            m for m in reversed(self._session_memory[message.session_id]) if isinstance(m, QuestReviewTask)
        )
        assert review_request is not None
        # Check if the quest is approved.
        if message.approved:
            # Publish the quest creation result.
            await self.publish_message(
                QuestCreationResult(
                    task=review_request.quest_creation_task,
                    quest=review_request.quest,
                    quest_steps=review_request.quest_steps,
                    review=message.review,
                ),
                topic_id=TopicId("default", self.id.key),
            )
            print("Quest Creation Result:")
            print("-" * 80)
            print(f"Task:\n{review_request.quest_creation_task}")
            print("-" * 80)
            print(f"Quest:\n{review_request.quest}")
            print("-" * 80)
            print(f"Quest Steps:\n{review_request.quest_steps}")
            print("-" * 80)
            print(f"Review:\n{message.review}")
            print("-" * 80)
        else:
            # Create a list of LLM messages to send to the model.
            messages: List[LLMMessage] = [*self._system_messages]
            for m in self._session_memory[message.session_id]:
                if isinstance(m, QuestReviewResult):
                    messages.append(UserMessage(content=m.review, source="Reviewer"))
                elif isinstance(m, QuestReviewTask):
                    messages.append(AssistantMessage(content=m.quest_creation_scratchpad, source="Quest_Creator"))
                elif isinstance(m, QuestCreationTask):
                    messages.append(UserMessage(content=m.task, source="User"))
                else:
                    raise ValueError(f"Unexpected message type: {m}")
            # Generate a revision using the chat completion API.
            response = await self._model_client.create(messages, cancellation_token=ctx.cancellation_token, extra_create_args={
                # "response_format": {"type": "json_schema", "json_schema": QuestCreatorResponse},
                "response_format": {"type": "json_object"}
            })
            assert isinstance(response.content, str)
            # Parse the response.
            parsed_response = QuestCreatorResponse.model_validate_json(response.content)
            # Create a new code review task.
            quest_review_task = QuestReviewTask(
                session_id=message.session_id,
                quest_creation_task=review_request.quest_creation_task,
                quest_creation_scratchpad=parsed_response.thoughts,
                quest=parsed_response.quest,
                quest_steps=parsed_response.steps,
            )
            # Store the code review task in the session memory.
            self._session_memory[message.session_id].append(quest_review_task)
            # Publish a new code review task.
            await self.publish_message(quest_review_task, topic_id=TopicId("default", self.id.key))


@default_subscription
class ReviewerAgent(RoutedAgent):
    """An agent that reviews quests."""

    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("A quest reviewer agent.")
        self._system_messages: List[LLMMessage] = [
            SystemMessage(
                content="""You are a quest reviewer. You analyzes and critiques each quest, highlighting potential issues with balance, narrative coherence, and player engagement.
                Respond using the following JSON format:
                {
                    "balance": "<Your comments>",
                    "coherence": "<Your comments>",
                    "approval": "<APPROVE or REVISE>",
                    "suggested_changes": "<Your comments>"
                }
                """,
            )
        ]
        self._session_memory: Dict[str, List[QuestReviewTask | QuestReviewResult]] = {}
        self._model_client = model_client

    @message_handler
    async def handle_quest_review_task(self, message: QuestReviewTask, ctx: MessageContext) -> None:
        # Format the prompt for the quest review.
        # Gather the previous feedback if available.
        previous_feedback = ""
        if message.session_id in self._session_memory:
            previous_review = next(
                (m for m in reversed(self._session_memory[message.session_id]) if isinstance(m, QuestReviewResult)),
                None,
            )
            if previous_review is not None:
                previous_feedback = previous_review.review
        # Store the messages in a temporary memory for this request only.
        self._session_memory.setdefault(message.session_id, []).append(message)
        prompt = f"""The problem statement is: {message.quest_creation_task}
            The quest is:
            ```
            {message.quest}
            ```

            The quest steps are:
            ```
            {message.quest_steps}
            ```

            Previous feedback:
            {previous_feedback}

            Please review the quest. If previous feedback was provided, see if it was addressed.
        """
        # Generate a response using the chat completion API.
        response = await self._model_client.create(
            self._system_messages
            + [UserMessage(content=prompt, source=self.metadata["type"])],
            cancellation_token=ctx.cancellation_token,
            extra_create_args={
                # "response_format": {"type": "json_schema", "json_schema": QuestReviewResponse},
                "response_format": {"type": "json_object"}
            },
        )
        assert isinstance(response.content, str)
        # Parse the response JSON.
        parsed_response = QuestReviewResponse.model_validate_json(response.content)
        review = parsed_response.model_dump()
        review_text = "Quest review:\n" + "\n".join([f"{k}: {v}" for k, v in review.items()])
        approved = parsed_response.approval.lower().strip() == "approve"
        result = QuestReviewResult(
            review=review_text,
            session_id=message.session_id,
            approved=approved,
        )
        # Store the review result in the session memory.
        self._session_memory[message.session_id].append(result)
        # Publish the review result.
        await self.publish_message(result, topic_id=TopicId("default", self.id.key))

## Logging

In [6]:
import logging

logging.basicConfig(level=logging.WARNING)
logging.getLogger("autogen_core").setLevel(logging.DEBUG)

## Testing the Reflection Design Pattern

In [7]:
from autogen_core import DefaultTopicId, SingleThreadedAgentRuntime
from autogen_ext.models.openai import OpenAIChatCompletionClient

LLAMA_70B_MODEL = "llama-3.3-70b-versatile"

model_client = OpenAIChatCompletionClient(
    model=LLAMA_70B_MODEL,
    temperature=0.0,
    api_key="<your_groq_api_key>",
    base_url="https://api.groq.com/openai/v1/",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "family": "unknown",
    },
)

runtime = SingleThreadedAgentRuntime()
await ReviewerAgent.register(
    runtime,
    "ReviewerAgent",
    lambda: ReviewerAgent(model_client=model_client),
)
await QuestCreatorAgent.register(
    runtime,
    "QuestCreatorAgent",
    lambda: QuestCreatorAgent(model_client=model_client),
)

runtime.start()

for i, character in enumerate(characters[5:6]): # update range of characters
    task_message = f"""
    Create an engaging quest for a character with the following details:

    CHARACTER NAME: {character["name"]}

    BACKSTORY:
    {character["backstory"]}

    QUEST FOCUS:
    {character["quest_focus"]}

    Please design a quest that:
    1. Aligns with the character's backstory and personal motivations
    2. Incorporates their key interests
    3. Follows the specified quest focus
    4. Has clear objectives and an interesting storyline
    5. Includes appropriate challenges that match the character's abilities
    6. Provides meaningful rewards that fit the character's interests
    """.strip()

    # Publish message
    print(
        f"Publishing task message for character: {character['name']}"
    )
    try:
        await runtime.publish_message(
            message=QuestCreationTask(
                task=task_message,
            ),
            topic_id=DefaultTopicId(),
        )
        print(
            f"Successfully published task for character: {character['name']}"
        )
    except Exception as e:
        print(
            f"Error publishing task for character {character['name']}: {str(e)}"
        )
        # Continue with other characters even if one fails
        continue

# Keep processing messages until idle.
await runtime.stop_when_idle()

INFO:autogen_core:Publishing message of type QuestCreationTask to all subscribers: {'task': "Create an engaging quest for a character with the following details:\n\n    CHARACTER NAME: Gandalf the Grey\n\n    BACKSTORY:\n    A wise and powerful wizard, Gandalf guides a fellowship on a quest to destroy the One Ring.\n\n    QUEST FOCUS:\n    Adventure and knowledge quests with a focus on magic, ancient lore, and epic battles.\n\n    Please design a quest that:\n    1. Aligns with the character's backstory and personal motivations\n    2. Incorporates their key interests\n    3. Follows the specified quest focus\n    4. Has clear objectives and an interesting storyline\n    5. Includes appropriate challenges that match the character's abilities\n    6. Provides meaningful rewards that fit the character's interests"}
INFO:autogen_core.events:{"payload": "{\"task\": \"Create an engaging quest for a character with the following details:\\n\\n    CHARACTER NAME: Gandalf the Grey\\n\\n    BACK

Publishing task message for character: Gandalf the Grey
Successfully published task for character: Gandalf the Grey


INFO:autogen_core.events:{"type": "LLMCall", "messages": [{"content": "You are a quest creator. You create quests for characters with different backgrounds, interests, and quest focuses.\n                You work with a quest giver who reviews your quest and provides feedback to improve it.\n                A quest is a task or mission that a player can complete by performing a series of steps.The quest must have following fields: {\n  \"$defs\": {\n    \"QuestDifficulty\": {\n      \"description\": \"Enum representing the difficulty level of a quest.\",\n      \"enum\": [\n        \"easy\",\n        \"medium\",\n        \"hard\",\n        \"epic\"\n      ],\n      \"title\": \"QuestDifficulty\",\n      \"type\": \"string\"\n    },\n    \"QuestType\": {\n      \"description\": \"Enum representing different types of quests available in the system.\\nEach type defines a distinct category of quest objectives and gameplay styles.\",\n      \"enum\": [\n        \"creative\",\n        \"know

Quest Creation Result:
--------------------------------------------------------------------------------
Task:
Create an engaging quest for a character with the following details:

    CHARACTER NAME: Gandalf the Grey

    BACKSTORY:
    A wise and powerful wizard, Gandalf guides a fellowship on a quest to destroy the One Ring.

    QUEST FOCUS:
    Adventure and knowledge quests with a focus on magic, ancient lore, and epic battles.

    Please design a quest that:
    1. Aligns with the character's backstory and personal motivations
    2. Incorporates their key interests
    3. Follows the specified quest focus
    4. Has clear objectives and an interesting storyline
    5. Includes appropriate challenges that match the character's abilities
    6. Provides meaningful rewards that fit the character's interests
--------------------------------------------------------------------------------
Quest:
title='The Lost Scrolls of Númenor' description='Retrieve the ancient scrolls containing